# Questions

* Should I calculate business days since incident? or Total days?
* How to do timeseries?
  * Only predict a date based on the info that's happened BEFORE it?
  * Book by week? exact day of year? month?
 ---
* Could I make an EITHER/OR case?
  * First do a classifier for Compensate or Deny. If compensate, then do a Regression for HOW MUCH they'll get back?
  * I could also do models that ONLY take into account how much they asked for, IF I have enough data
* Why is my RF 410 MB?
* **I may be parsing 'ITEMS' wrong. Semicolon may indicate the various items missing**

### 1st simple model, "settle" or "compensate" I got 67.40% ROC!
### 2nd model, with claim_request_amount, roc_auc got 68.64%

# Next Steps
* Try a Rolling Window of 12 months to remove seasonality and see if there are changes from 2002 onward
* See if I can spot trends by day/week/month/holidays/weekends during the year

### *Is it an issue with 370 airports and 170 airlines?*

# Feature engineer count of how many claims

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import pyspark

import datetime as dt

import tabula
import joblib

In [202]:
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 60)
pd.set_option('display.precision', 3)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import sklearn
from sklearn.preprocessing import StandardScaler, Binarizer, LabelBinarizer, MultiLabelBinarizer, OneHotEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score, cross_validate \
                                    ,cross_val_predict, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix,recall_score,precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

from sklearn.metrics import roc_curve, auc

import itertools
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib
from imblearn.over_sampling import SMOTE

In [22]:
dir = 'models/preliminary'

-----

In [23]:
# # A function that does the cleaning if you pass the df

# def colnames_dt_drops_dtypes(df):
#     col_names = ['claim_number', 'date_received', 'incident_date', 'airport_code', 'airport_name',
#            'airline', 'claim_type', 'claim_site', 'item_category', 'close_amount', 'disposition']
#     df['date_received'] = pd.to_datetime(df['date_received'])
#     df['incident_date'] = pd.to_datetime(df['incident_date'])
#     df = df[df['disposition'] != "-"]
#     df = df.dropna()
#     df['claim_number'] = df['claim_number'].astype('int64')
#     df['close_amount'] = df['close_amount'].astype('int64')
#     df['binary_disposition'] = df['disposition']
#     df['binary_disposition'] = df['binary_disposition'].where(df['binary_disposition'] == 'Deny', other='Compensate')
#     return df

-----

In [24]:
col_names = ['claim_number', 'date_received', 'incident_date', 'airport_code', 'airport_name',
           'airline', 'claim_type', 'claim_site', 'item_category', 'close_amount', 'disposition']
           

In [190]:
df = pd.read_excel('raw/claims-2010-2013.xls', names=col_names)

df['date_received'] = pd.to_datetime(df['date_received'])
df['incident_date'] = pd.to_datetime(df['incident_date'])
df = df[df['disposition'] != "-"]
df = df.dropna()

df['claim_number'] = df['claim_number'].astype('int64')
df['close_amount'] = df['close_amount'].astype('int64')
df['binary_disposition'] = df['disposition']
df['binary_disposition'] = df['binary_disposition'].where(df['binary_disposition'] == 'Deny', other='Compensate')

# Change some text to make it more human readable
df.claim_site[df.claim_site == '-'] = 'Unknown'
df.claim_type[df.claim_type == '-'] = 'Unknown'

# Feature Engineering

### Count of items claimed

# TODO: Consider only using this for where claim_type is related to property.
df['num_items_or_incidents_claimed'] = df['item_category'].str.split(pat=';').apply(lambda x: len(x))
df['num_items_or_incidents_claimed'] = df['num_items_or_incidents_claimed'].where(df['claim_type'].str.contains('property', case=False) == True, other= 0)

### Time calculation
wait_period = df.date_received - df.incident_date
df['days_waited_to_file_claim'] = wait_period.dt.days

# Drop days where the 'date_received" was reported before 'incident_date'
df = df[df.days_waited_to_file_claim >= 0]

df.reset_index(inplace=True,drop=True)

/home/spencer/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/spencer/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [197]:
df.head()

,claim_number,date_received,incident_date,airport_code,airport_name,airline,claim_type,claim_site,item_category,close_amount,disposition,binary_disposition,num_items_or_incidents_claimed,days_waited_to_file_claim
0,2010010567154,2010-01-04,2010-01-03 14:30:00,SLC,Salt Lake City International Airport,Delta Air Lines,Property Damage,Checked Baggage,Cosmetics & Grooming,0,Deny,Deny,1,0
1,2010010567158,2010-01-04,2010-01-02 00:00:00,LAX,Los Angeles International Airport,Southwest Airlines,Passenger Property Loss,Checked Baggage,Other,0,Deny,Deny,1,2
2,2010010567173,2010-01-04,2010-01-02 05:00:00,SEA,Seattle-Tacoma International,Delta Air Lines,Passenger Property Loss,Checked Baggage,Cameras; Cameras,0,Deny,Deny,2,1
3,2010010567182,2010-01-04,2010-01-02 00:00:00,LAS,McCarran International,American Airlines,Passenger Property Loss,Checked Baggage,Travel Accessories,0,Deny,Deny,1,2
4,2010010567184,2010-01-04,2010-01-03 00:00:00,DFW,Dallas-Fort Worth International Airport,American Airlines,Passenger Property Loss,Checked Baggage,Travel Accessories,0,Deny,Deny,1,1


In [ ]:
df.sort_values(by='close_amount', ascending=False).head(3)

In [28]:
# df_simple = df.drop(['claim_number', 'airport_name', 
#                      #'binary_disposition',
#                      'disposition', 
#                      'date_received', 'incident_date'
#                     ]
#                     , axis=1)

In [191]:
X = df.drop(['claim_number', 'airport_name', 'binary_disposition', 'disposition', 
             'date_received', 'incident_date',
             'close_amount',
            ]
            , axis=1)
y = df['binary_disposition'].apply(lambda x: 1 if x == 'Compensate' else 0)

In [192]:
categorical = ['airport_code', 'airline', 'claim_type', 'claim_site']
continuous =  ['days_waited_to_file_claim', 'num_items_or_incidents_claimed']

In [193]:
enc = OneHotEncoder(sparse=False)
onehotarray = enc.fit_transform(X[categorical])

ss = StandardScaler()
continuousarray = ss.fit_transform(X[continuous])

mlb = MultiLabelBinarizer(sparse_output=False)
onehot_itemcategories = mlb.fit_transform(X['item_category'].str.replace(' ','').str.split(pat=';'))

X = np.concatenate((onehotarray, continuousarray, onehot_itemcategories), axis=1)

/home/spencer/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/spencer/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [194]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [195]:
X.shape

(34478, 606)

In [196]:
rf_clf = RandomForestClassifier(n_estimators=100, criterion='gini', n_jobs=5, random_state=42)
cvs = cross_val_score(rf_clf, X_train, y_train, cv=10, scoring='roc_auc')
print(np.mean(cvs))

0.6814186309902636


In [199]:
rf_clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=5,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [203]:
y_predict = rf_clf.predict(X_test)

roc_auc_score(y_test, y_predict)

In [ ]:
# make a schedule from now til Thursday
# make change max_depth to [3, 5, 10] to see which works best if i am overfitting

In [204]:
rf = RandomForestClassifier(random_state=42)

criterions = ['gini']#, 'entropy']
n_ests = [100, 300]
m_depths = [3, 5, 10, 15]
    
param_grid = dict(criterion=criterions, n_estimators=n_ests, max_depth=m_depths)

grid_rf = GridSearchCV(rf, param_grid, scoring='roc_auc', cv=8, n_jobs=4)

grid_rf.fit(X_train, y_train)

print(grid_rf.best_score_)
print(grid_rf.best_params_)
print(grid_rf.best_estimator_)

y_pred = grid_rf.predict(X_test)
print(roc_auc_score(y_test, y_pred))

0.7220751078811192
{'criterion': 'gini', 'max_depth': 15, 'n_estimators': 300}
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=15, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)
0.5953619560289024


In [207]:
rf = RandomForestClassifier(random_state=42)

criterions = ['gini']#, 'entropy']
n_ests = [100, 300]
m_depths = [20, 30, 40, 50]
    
param_grid = dict(criterion=criterions, n_estimators=n_ests, max_depth=m_depths)

grid_rf2 = GridSearchCV(rf, param_grid, scoring='roc_auc', cv=8, n_jobs=4)

grid_rf2.fit(X_train, y_train)

print(grid_rf2.best_score_)
print(grid_rf2.best_params_)
print(grid_rf2.best_estimator_)

y_pred2 = grid_rf2.predict(X_test)
print(roc_auc_score(y_test, y_pred2))

/home/spencer/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/home/spencer/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/home/spencer/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/home/spencer/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/ex

0.725709095769306
{'criterion': 'gini', 'max_depth': 20, 'n_estimators': 300}
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)
0.6062968483199722


In [206]:
rf = RandomForestClassifier(random_state=42)

criterions = ['gini']#, 'entropy']
n_ests = [300]
m_depths = [10, 15, 20]
    
param_grid = dict(criterion=criterions, n_estimators=n_ests, max_depth=m_depths)

grid_rf3 = GridSearchCV(rf, param_grid, scoring='roc_auc', cv=8, n_jobs=4)

grid_rf3.fit(X_train, y_train)

print(grid_rf3.best_score_)
print(grid_rf3.best_params_)
print(grid_rf3.best_estimator_)

y_pred3 = grid_rf3.predict(X_test)
print(roc_auc_score(y_test, y_pred3))

0.725709095769306
{'criterion': 'gini', 'max_depth': 20, 'n_estimators': 300}
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)
0.6062968483199722


In [516]:
dir = 'models/preliminary'
joblib.dump(grid_rf.best_estimator_, f'{dir}/rf_without_dates_or_requested_amount', compress=9)

['models/preliminary/rf_without_dates_or_requested_amount']